**IMPORTING THE NECESSARY LIBRARIES**

In [5]:
from torch.utils.data import DataLoader, Dataset
import numpy as np 
import pandas as pd 
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

**CREATING THE DATASET**

In [6]:
class Dataset(Dataset):
    def __init__(self):
        self.filepath ='/kaggle/input/car-steering-angle-prediction/driving_dataset/'
        self.filenames = os.listdir(self.filepath)
    
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self,index):
        filename = self.filenames[index]
        img = cv2.imread(self.filepath+filename)
        
        resized = cv2.resize(img, (66,200),interpolation=cv2.INTER_AREA)
        return torch.from_numpy(resized.transpose()).float(), torch.rand(1)


**LOADING DATA**

In [7]:
data = Dataset()
data_loader = torch.utils.data.DataLoader(data,batch_size=32 ,shuffle=True )


In the data ,there are tuples whch its first elemnt is tensor and second element is angle. We will try to predict the angle by using tensors which represents the pictures.

In [8]:
print('Dimension of Images',data[0][0].shape)
print('numberofsample',len(data))

In [9]:
print('Numerical Representations of Images',data[0][0],'\n')
print('Angle of Car Steel on The image', data[0][1])

**SPLITTING INTO TEST TRAIN AND VALIDATION**

In [10]:
train =list(range(0,30000))
test = list(range(30000,40000))
valid =list(range(40000,45570))

In [11]:
trainset = torch.utils.data.Subset(data, train)
testset = torch.utils.data.Subset(data, test)
validset=torch.utils.data.Subset(data, valid)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,shuffle=True)
testloader= torch.utils.data.DataLoader(testset, batch_size=32,shuffle=True)
validloader= torch.utils.data.DataLoader(validset, batch_size=32,shuffle=True)


In [12]:
print('Train Set Samples:',len(trainset))
print('Test Set Samples',len(testset))
print('Valid Set Samples',len(validset))

In [13]:
trainset[0][0].shape

**CONVOLUTIONAL MODEL**

In [14]:
class paper_model(nn.Module):
    def __init__(self):
        super(paper_model, self).__init__()

        self.conv_layers = nn.Sequential(
            
            #conv2d pytrochs implementation of convolutional layers
            nn.Conv2d(3, 24, kernel_size=5, stride=2),
            nn.ELU(),
            
            nn.Conv2d(24, 36, kernel_size=5, stride=2),
            nn.ELU(),
            
            nn.Conv2d(36, 48, kernel_size=5, stride=2),
            nn.ELU(),
            
            nn.Conv2d(48, 64, kernel_size=3, stride=1),
            nn.ELU(),
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ELU(),
            
            nn.Dropout(p=0.5)
        )
        
        self.linear_layers = nn.Sequential(
            
            #linear fully connected layers
            nn.Linear(in_features=64*1*18, out_features=100),
            nn.ELU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(in_features=100, out_features=64),
            nn.ELU(),
            
            nn.Linear(in_features=64, out_features=10),
            nn.ELU(),
            
            nn.Linear(in_features=10, out_features=1)
        )
        
    
    def forward(self, x):
        #input image size:
        x = x.view(x.size(0), 3, 66, 200)
        output = self.conv_layers(x)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output
 

**SETTING OPTIMIZATION**

In [15]:
net = paper_model()
optimizer = torch.optim.Adam(net.parameters(),lr =1e-3)
criterion = torch.nn.MSELoss()

**TRAINING**

In [16]:

image_loss = []
for i, sample in enumerate(trainloader):
    optimizer.zero_grad()
    prediction = net(sample[0])
    loss = criterion(prediction.squeeze() , torch.rand(1))
    print(f"{i}th loss: ", loss.item())
    image_loss.append(loss)
    loss.backward()
    optimizer.step()

**RMSE**

In [ ]:
import matplotlib.pyplot as plt

In [28]:
rmse = []
for i in image_loss:
    rmse.append(i**0.5)

In [31]:
plt.title('LOSS FOR EACH IMAGE')
plt.xlabel('Image Number')
plt.ylabel('RMSE')
plt.plot(rmse)
plt.show()

**TOTAL RMSE**

In [36]:
print('Total RMSE:', sum(rmse))